In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import cv2
import glob

In [3]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [4]:
Autoencoder = tf.keras.models.load_model('autoencoder-model_justCSA3.h5')
Autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 16)                4352      
_________________________________________________________________
repeat_vector (RepeatVector) (None, 90, 16)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 90, 8)             800       
_________________________________________________________________
time_distributed (TimeDistri (None, 90, 51)            459       
Total params: 5,611
Trainable params: 5,611
Non-trainable params: 0
_________________________________________________________________


In [5]:
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [11]:
cap = cv2.VideoCapture('/Users/danukathaja/Downloads/Flask-Web-Framework-main/Tutorial 7/20220325_192234.mp4')
list =[]
frames_list=[]
#name = video + '.avi'
out = cv2.VideoWriter('/Users/danukathaja/Downloads/Flask-Web-Framework-main/Tutorial 7/output2.avi' , cv2.VideoWriter_fourcc('M','J','P','G'), 30.0, (640,352))

while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = img[680:2000, 400:640]
    h, w, c = img.shape
    #print(h,w)
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 224,224)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores_v = results['output_0'].numpy()[:,0,:51].reshape((1,17,3))
    keypoints_with_scores_A = results['output_0'].numpy()[:,0,:51].reshape(51)
    list.append(keypoints_with_scores_A)
    data = np.array(list)
    a,b = data.shape
    #frame = frame[680:2000, 400:640]

    
    # Render keypoints 
    #loop_through_people(frame, keypoints_with_scores_v, EDGES, 0.1)

    if a < 120:
        loop_through_people(frame, keypoints_with_scores_v, EDGES, 0.9)
        #print('waiting')
    else:
        #cv2.putText(frame,'Infrances Strated',(50, 100),cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 0, 0),2,cv2.LINE_4)
        data1 = data[a-90:a,:]
        data1 = np.expand_dims(data1, axis=0)
        #print(data1.shape)
        pred = Autoencoder.predict(data1)
        #print()
        pred = pred[0,89,:].reshape((1,17,3))
        loop_through_people(frame, keypoints_with_scores_v, EDGES, 1)
        evl = Autoencoder.evaluate(data1,data1)
        dis = evl
        if dis > 0.0135:
            cv2.putText(frame,'Drunk',(50, 50),cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),2,cv2.LINE_4)
        else:
            cv2.putText(frame,'Not Drunk',(50, 50),cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),2,cv2.LINE_4)
        print(evl)
    frames_list.append(frame)
    out.write(frame)
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 3ms/step - loss: 0.0091
0.009099732153117657
1/1 [==============================] - 0s 2ms/step - loss: 0.0107
0.01070642564445734
1/1 [==============================] - 0s 2ms/step - loss: 0.0122
0.012244855985045433
1/1 [==============================] - 0s 3ms/step - loss: 0.0120
0.011952933855354786
1/1 [==============================] - 0s 1ms/step - loss: 0.0111
0.011122401803731918
1/1 [==============================] - 0s 3ms/step - loss: 0.0109
0.01093018613755703
1/1 [==============================] - 0s 1ms/step - loss: 0.0094
0.00937369279563427
1/1 [==============================] - 0s 1ms/step - loss: 0.0092
0.009166067466139793
1/1 [==============================] - 0s 2ms/step - loss: 0.0098
0.009829691611230373
1/1 [==============================] - 0s 2ms/step - loss: 0.0114
0.011407033540308475
1/1 [==============================] - 0s 1ms/step - loss: 0.0143
0.014257102273404598
1/1 [==============================] - 0s 2ms/

AttributeError: 'NoneType' object has no attribute 'copy'